In [67]:
import sys

sys.argv = [""]
del sys

In [68]:
import argparse
import os
from copy import deepcopy

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# causing the kernal to die.!!
# from inpaint.data import *
from inpaint.data import PlacesDataset 
# from inpaint.utils import random_bbox_mask, random_ff_mask
# from inpaint.utils.metrics import psnr
# from inpaint.utils.metrics import ssim

import matplotlib.pyplot as plt

import random

In [69]:
def flipCoin():
    f = random.random()
    return True if f < 0.5 else False

In [70]:
def create_cfg(args):
    parser = argparse.ArgumentParser()
    for key, val in args.items():
        parser.add_argument(key, default=val)

    cfg = parser.parse_args()
    return cfg

In [71]:
args = {}
# model
args["--in_channels"] = 4
args["--out_channels"] = 3
args["--latent_channels"] = 64
args["--pad_type"] = "zero"
args["--activation"] = "elu"
args["--norm_d"] = "none"
args["--norm_g"] = "none"
args["--init_type"] = "kaiming"
args["--init_gain"] = 0.02
args["--use_cuda"] = False

# dataset
args["--val_ds_dir"] = '../samples/Places365'
args["--crop_size"] = (256, 256)
args["--batch_size"] =  1
args["--num_workers"] = 1
args["--device_id"] = 0
args["--val_steps"] = 10

In [72]:
cfg = create_cfg(args)

In [73]:
val_ds = PlacesDataset(path_dir = cfg.val_ds_dir, transform_config=('to_tensor', 'center_crop', 'norm'), crop_size=cfg.crop_size)
val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=True, pin_memory=True, num_workers=cfg.num_workers)

In [74]:
print(f"Total validation images: {len(val_ds)}")

Total validation images: 24


___

In [75]:
from inpaint.core.generator import GatedGenerator
from inpaint.tools import Evaluate
import glob

___

1. Initialize generator from a saved checkpoint

In [78]:
# load checkpoint and initialize
checkpoints_dir= '../../checkpoints/exp1'

checkpointslist= (glob.glob("../../checkpoints/exp1/*.pth")) 

# checkpointslist = os.listdir(checkpoints_dir)
print(checkpointslist)

best_model_index = checkpointslist.index('../../checkpoints/exp1/best_models_final.pth')

best_model = checkpointslist[best_model_index]
# print(best_model)

val_steps = cfg.val_steps
generator = GatedGenerator(cfg)
# generator = None
map_location=torch.device('cpu')
models = torch.load(best_model, map_location=map_location )
gen_model_state_dict = models["generator_state_dict"]
generator = generator.load_state_dict(gen_model_state_dict)

# generator.to(self.device)
# generator.eval()

['../../checkpoints/exp1/best_models_final.pth', '../../checkpoints/exp1/ckpt_epoch1.pth', '../../checkpoints/exp1/ckpt_epoch30.pth']


<All keys matched successfully>

___

2. Create Evaluate instance pass dataloader and generator instance to Evaluate 

In [79]:
eval = Evaluate(generator,val_loader)

AttributeError: 'GatedGenerator' object has no attribute 'device_id'

____
3. Evaluate (return avg psnr and avg ssim value

In [34]:
avg_psnr, avg_ssim = eval.evaluate()

AttributeError: 'builtin_function_or_method' object has no attribute 'evaluate'